In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import re

In [3]:
#각 지역마다 필요한 열만 추출

path = './files/'
files = os.listdir(path)

cs = pd.DataFrame()

for file in files:
    data = pd.read_excel(path + file, usecols ='A,C,E')
    cs = pd.concat([cs,data])
    
cs_total = cs.reset_index(drop = True)

In [4]:
#결측값 확인
cs_total.isna().sum()

시도교육청    0
폐교년도     0
급별       0
dtype: int64

In [5]:
#nan 결측치 대체 (~초등 / ~중등 / ~고등)

for i in range(len(cs_total)):
    if cs_total['급별'].isna()[i] == True:
        
        if "초등" in cs_total['폐교명'][i]:
            cs_total['급별'].values[i] = '초'
            
        elif "중등" in cs_total['폐교명'][i] == True:
            cs_total['급별'].values[i] = '중'
            
        elif "고등" in cs_total['폐교명'][i] == True:
            cs_total['급별'].values[i] = '고'

In [6]:
#nan 결측치 대체 (~초 / ~중 / ~고)

for i in range(len(cs_total)):
    if cs_total['급별'].isna()[i] == True:
        
        str = cs_total["폐교명"][i]
        print(str)
        
        if str[-1] == "초":
            cs_total['급별'].values[i] = '초'
            
        elif str[-1] == '중':
            cs_total['급별'].values[i] = '중'
            
        elif str[-1] == '고':
            cs_total['급별'].values[i] = '고'

In [7]:
cs_total

,시도교육청,폐교년도,급별
0,부산,2009,중
1,부산,2007,초
2,부산,2010,고
3,부산,2016,초
4,부산,2020,중
...,...,...,...
1359,울산,1999,초
1360,울산,1999,초
1361,울산,1999,초
1362,울산,2016,초


In [ ]:
#필요한 Column만 추출한 엑셀파일 저장
cs_total.to_excel('./files/cs_total.xlsx',index=False)

In [8]:
#필요 데이터만 추출
cs_close = (cs_total["폐교년도"] >= 2001) & (cs_total["폐교년도"] <= 2020)
cs_totals = cs_total[cs_close]

In [9]:
#폐교년도 타입 변경
cs_totals["폐교년도"] = cs_totals["폐교년도"].astype("str")

C:\Users\Doffal\AppData\Local\Temp/ipykernel_20648/809716553.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cs_totals["폐교년도"] = cs_totals["폐교년도"].astype("str")


In [10]:
#인덱스 리셋
cs_totals = cs_totals.reset_index(drop =True)

In [11]:
cs_totals.head()

,시도교육청,폐교년도,급별
0,부산,2009,중
1,부산,2007,초
2,부산,2010,고
3,부산,2016,초
4,부산,2020,중


In [ ]:
cs_totals.to_excel('./cs_totals.xlsx')

In [ ]:
cs_all = cs_totals.drop(['급별'], axis = 'columns')

In [ ]:
cs_all = cs_all.groupby(('폐교년도'),  as_index =False).count()

In [ ]:
#시각화 
cs_all= cs_all.rename(columns = {'시도교육청':'폐교 수'})

In [ ]:
#상관분석을 위한 파일 저장
cs_all.to_excel('./cs_all.xlsx')

In [ ]:
#시계열그래프 시각화_1

cs_all.plot(figsize = (20, 10), kind = 'line', x ='폐교년도', y ='폐교 수', color ='orange')

plt.show()

plt.savefig('./closed_school')

In [ ]:
#시계열그래프 시각화

#그래프 크기
plt.figure(figsize = (30, 15))

#그래프 x 및 y행에 대한 데이터 지정
plt.plot(cs_all['폐교년도'], cs_all['폐교 수'], color = 'orange', label='Closed School')

#x 눈금레이블 크기
plt.xticks(fontsize = 20)

#y 눈금레이블 크기
plt.yticks(fontsize = 20)

#x레이블 제목 크기 
plt.xlabel('year', fontsize = 30)

#y레이블 제목 크기
plt.ylabel('count', fontsize = 30)

#그림 제목 크기
plt.title('Closed School', fontsize = 30)

#범례위치 및 크기 지정
plt.legend(loc = 'upper left', fontsize = 15)

plt.savefig('./graph_closeschool/Close_School_all')

# 초 중 고등별 폐교데이터

In [12]:
time = ['2001', '2002', '2003', '2004', '2005', 
        '2006', '2007', '2008', '2009', '2010', 
        '2011', '2012', '2013', '2014', '2015', 
        '2016', '2017', '2018', '2019', '2020']

In [13]:
time_dic = {'폐교년도' : time}

In [14]:
#DataFrame형식 변환
time_df = pd.DataFrame(time_dic)

In [15]:
time_df

,폐교년도
0,2001
1,2002
2,2003
3,2004
4,2005
5,2006
6,2007
7,2008
8,2009
9,2010


In [22]:
cs_totals.head()

,시도교육청,폐교년도,급별
0,부산,2009,중
1,부산,2007,초
2,부산,2010,고
3,부산,2016,초
4,부산,2020,중


In [17]:
cs_emh = cs_totals.groupby(['폐교년도', '급별'], as_index =False).count()

In [19]:
cs_emh.head()

,폐교년도,급별,시도교육청
0,2001,고,2
1,2001,중,2
2,2001,초,24
3,2002,중,3
4,2002,초,23


In [20]:
ele = (cs_emh['급별'] == '초')
cs_ele = cs_emh[ele]

In [26]:
cs_ele.head()

,폐교년도,급별,시도교육청
2,2001,초,24
4,2002,초,23
6,2003,초,9
9,2004,초,13
12,2005,초,16


In [23]:
#left join
cs_ele_1 = pd.merge(time_df, cs_ele, how='left')
cs_ele_1 = cs_ele_1.fillna(0)

In [24]:
cs_ele_2 = cs_ele_1.drop(['폐교년도' ,'급별'], axis=1)

In [25]:
cs_ele_2

,시도교육청
0,24
1,23
2,9
3,13
4,16
5,9
6,38
7,27
8,34
9,25


In [28]:
#중학교
mid = (cs_emh['급별'] == '중')
cs_mid = cs_emh[mid]

In [29]:
#left join
cs_mid_1 = pd.merge(time_df, cs_mid, how='left')
cs_mid_1 = cs_mid_1.fillna(0)

In [30]:
cs_mid_2 = cs_mid_1.drop(['폐교년도' ,'급별'], axis=1)

In [ ]:
cs_mid_2

In [31]:
hig = (cs_emh['급별'] == '고')
cs_hig = cs_emh[hig]

In [32]:
#left join
cs_hig_1 = pd.merge(time_df, cs_hig, how='left')
cs_hig_1 = cs_hig_1.fillna(0)

In [33]:
cs_hig_2 = cs_hig_1.drop(['폐교년도' ,'급별'], axis=1)

In [34]:
cs_all_emh = pd.concat([time_df, cs_ele_2, cs_mid_2, cs_hig_2], axis=1)

In [35]:
cs_all_emh.columns = ['year', 'elementary', 'middle', 'high']

In [37]:
cs_all_emh['all'] = cs_all_emh['elementary'] + cs_all_emh['middle'] + cs_all_emh['high']

In [39]:
cs_all_emh.head()

,year,elementary,middle,high,all
0,2001,24,2.0,2.0,28.0
1,2002,23,3.0,0.0,26.0
2,2003,9,0.0,1.0,10.0
3,2004,13,2.0,1.0,16.0
4,2005,16,4.0,1.0,21.0


In [ ]:
cs_all_emh.to_excel('./cs_all_emh.xlsx')